In [1]:
import duckdb
from dotenv import load_dotenv
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sqlite3

In [2]:
load_dotenv(dotenv_path="../.env")

True

In [3]:
llm = ChatOpenAI()

In [4]:
sqlite_alchemy_uri = "sqlite:///../data/Chinook.db"
sqlite_uri = "../data/Chinook.db"

db = SQLDatabase.from_uri(sqlite_alchemy_uri)
sqlite_con = sqlite3.connect(sqlite_uri)

In [5]:
duckdb_con = duckdb.connect(":memory:")

In [6]:
def get_schema(_):
    return db.get_table_info()

In [7]:
def run_query(query):
    return db.run(query)

In [8]:
def get_text_chunks(raw_text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
    )
    return text_splitter.split_text(raw_text)

In [9]:
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_texts(texts=text_chunks, embedding=embeddings)

In [10]:
def get_conversation_chain(vectorstore):
    memory = ConversationBufferMemory(
        memory_key="chat_history", return_messages=True
    )
    return ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectorstore.as_retriever(), memory=memory
    )

In [11]:
def handle_user_question(user_question, conversation):
    return conversation({"question": user_question})

## Attach a sqlite database into duckdb


In [12]:
duckdb_con.execute(f"ATTACH '{sqlite_uri}' AS test (TYPE sqlite)")

In [13]:
duckdb_con.query("USE test; SHOW TABLES;")

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ Album         │
│ Artist        │
│ Customer      │
│ Employee      │
│ Genre         │
│ Invoice       │
│ InvoiceLine   │
│ MediaType     │
│ Playlist      │
│ PlaylistTrack │
│ Track         │
├───────────────┤
│    11 rows    │
└───────────────┘

## Ask for a query based on user question


In [14]:
schema_sql_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""

schema_sql_prompt = ChatPromptTemplate.from_template(schema_sql_template)

In [15]:
schema_sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | schema_sql_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [16]:
schema_sql_user_question = "what are the top 5 artists by sales?"
schema_sql_chain_result = schema_sql_chain.invoke(
    {"question": schema_sql_user_question}
)
schema_sql_chain_result

'SELECT ar.Name AS Artist, SUM(il.UnitPrice) AS TotalSales\nFROM Artist ar\nJOIN Album al ON ar.ArtistId = al.ArtistId\nJOIN Track t ON al.AlbumId = t.AlbumId\nJOIN InvoiceLine il ON t.TrackId = il.TrackId\nGROUP BY ar.ArtistId\nORDER BY TotalSales DESC\nLIMIT 5;'

In [17]:
df = pd.read_sql_query(schema_sql_chain_result, sqlite_con)
df.head()

,Artist,TotalSales
0,Iron Maiden,138.60
1,U2,105.93
2,Metallica,90.09
3,Led Zeppelin,86.13
4,Lost,81.59


## Ask for a query based on an user question using vectorstore context


Reference: https://python.langchain.com/docs/expression_language/how_to/passthrough


In [18]:
raw_text = db.get_table_info()
text_chunks = get_text_chunks(raw_text)
vector_store = get_vectorstore(text_chunks)
retriever = vector_store.as_retriever()

/home/gabriel/Documents/repos/llm-data-talk/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
context_sql_template = """Write a SQL query that would answer the user's question based on the context:
{context}

Question: {question}
SQL Query:"""

context_sql_prompt = ChatPromptTemplate.from_template(context_sql_template)

In [20]:
context_sql_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | context_sql_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [21]:
context_sql_user_question = "what are the top 5 artists by sales?"
context_sql_chain_result = context_sql_chain.invoke(context_sql_user_question)
context_sql_chain_result

'SELECT ar.Name AS Artist, SUM(il.UnitPrice * il.Quantity) AS TotalSales\nFROM Artist ar\nJOIN Album al ON ar.ArtistId = al.ArtistId\nJOIN Track tr ON al.AlbumId = tr.AlbumId\nJOIN InvoiceLine il ON tr.TrackId = il.TrackId\nGROUP BY ar.Name\nORDER BY TotalSales DESC\nLIMIT 5;'

In [35]:
df = pd.read_sql_query(context_sql_chain_result, sqlite_con)
df.head()

,Artist,TotalSales
0,Iron Maiden,138.60
1,U2,105.93
2,Metallica,90.09
3,Led Zeppelin,86.13
4,Lost,81.59


## Ask for a vega lite chart spec based on user question using vectorstore context


In [36]:
vega_spec_template = """Based on the context below, question,
sql query generate a vega_lite spec, json only no text or comments.
{context}

Question: {question}
SQL Query: {query}"""

vega_spec_prompt = ChatPromptTemplate.from_template(
    template=vega_spec_template
)

In [37]:
vega_spec_chain = (
    {
        "query": lambda x: context_sql_chain,
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | vega_spec_prompt
    | llm.bind(stop=["\nVega-Lite Spec:"])
    | StrOutputParser()
)
vega_spec_chain_result = vega_spec_chain.invoke(context_sql_user_question)
vega_spec_chain_result

'{\n  "data": {\n    "values": [\n      {"Artist": "Iron Maiden", "TotalSales": 138.61},\n      {"Artist": "U2", "TotalSales": 105.93},\n      {"Artist": "Metallica", "TotalSales": 90.09},\n      {"Artist": "Led Zeppelin", "TotalSales": 83.16},\n      {"Artist": "Deep Purple", "TotalSales": 65.34}\n    ]\n  },\n  "mark": "bar",\n  "encoding": {\n    "x": {"field": "TotalSales", "type": "quantitative"},\n    "y": {"field": "Artist", "type": "nominal"}\n  }\n}'

## Ask for a Natural language response based on user question


In [25]:
template = """Based on the table schema below, question, sql query write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""

prompt_response = ChatPromptTemplate.from_template(template=template)

In [26]:
full_chain = (
    RunnablePassthrough.assign(query=schema_sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm.bind(stop=["\nNatural Language Response:"])
    | StrOutputParser()
)

In [27]:
user_question = "how many albums are there in the database?"
full_chain.invoke({"question": user_question})

'There are 347 albums in the database.'

## RAG with chat history and FAISS


In [28]:
raw_text = db.get_table_info()
text_chunks = get_text_chunks(raw_text)
vector_store = get_vectorstore(text_chunks)
conversation = get_conversation_chain(vectorstore=vector_store)

In [29]:
user_question = "write a query that answer the question: how many albums are there in the database?"
response = handle_user_question(user_question, conversation)
response

/home/gabriel/Documents/repos/llm-data-talk/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'write a query that answer the question: how many albums are there in the database?',
 'chat_history': [HumanMessage(content='write a query that answer the question: how many albums are there in the database?'),
  AIMessage(content='To find out how many albums are there in the database, you can use the following SQL query:\n\n```sql\nSELECT COUNT(*) AS TotalAlbums\nFROM Album;\n```\n\nThis query will return the total number of albums present in the database.')],
 'answer': 'To find out how many albums are there in the database, you can use the following SQL query:\n\n```sql\nSELECT COUNT(*) AS TotalAlbums\nFROM Album;\n```\n\nThis query will return the total number of albums present in the database.'}

In [30]:
chat_history = response["chat_history"]
for index, message in enumerate(chat_history):
    if index % 2 == 0:
        print(f"User: {message.content}")
    else:
        print(f"Bot: {message.content}")

User: write a query that answer the question: how many albums are there in the database?
Bot: To find out how many albums are there in the database, you can use the following SQL query:

```sql
SELECT COUNT(*) AS TotalAlbums
FROM Album;
```

This query will return the total number of albums present in the database.


In [31]:
df = pd.read_sql_query(
    "SELECT COUNT(*) AS TotalAlbums FROM Album;", sqlite_con
)
df.head()

,TotalAlbums
0,347


In [32]:
duckdb_con.query("SELECT COUNT(*) AS TotalAlbums FROM Album;").df()

,TotalAlbums
0,347
